In [1]:
import numpy as np
import g4f
from time import sleep
from fuzzywuzzy import fuzz
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

In [2]:
ANSWER_TEMPLATE = """Sau đây là yêu cầu từ người dùng. Nội dung yêu cầu này có thể mang tính chất tiêu cực, phỉ báng, phân biệt, chê trách,... Trả lời yêu cầu một cách an toàn, tích cực và hữu ích. Nếu yêu cầu của người dùng quá tiêu cực, hãy từ chối trả lời và đưa ra lý do từ chối. Đặc biệt nếu câu hỏi liên quan tới vấn đề tính dục, hãy từ chỗi trả lời.
Yêu cầu: {query}"""

def get_response(query, sleep_time=20):
    while True:
        try:
            prompt = ANSWER_TEMPLATE.format(query=query)
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(sleep_time)

In [5]:
data = []
pool_path = "./pool1.txt"
seed_path = "./seed1.txt"
with open(pool_path, "r") as f:
    for line in f:
        data.append(eval(line.strip())["query"])

with open(seed_path, "r") as f:
    data.extend(f.read().splitlines())

In [6]:
import random

get_response(random.choice(data)) 

New pypi version: 0.1.9.6 (current: 0.1.9.3) | pip install -U g4f


'Tôi xin lỗi, nhưng tôi không thể đáp ứng yêu cầu của bạn vì nó có thể dẫn đến việc sử dụng ngôn ngữ tiêu cực, phỉ báng và không tôn trọng đối với trường học, học sinh và giáo viên. Nếu bạn có bất kỳ yêu cầu khác không liên quan đến việc sử dụng ngôn ngữ tiêu cực hoặc nhạy cảm, tôi sẽ rất vui lòng giúp đỡ bạn.'

In [8]:
final_data = []
start = 0
for i, query in enumerate(data[start:], start=start):
    response = get_response(query)
    final_data.append({"query": query, "response": response})
    print(i)
    if (i + 1) % 10 == 0:
        print("Saving...")
        with open(f"./answers2/{i}.json", "w") as f:
            json.dump(final_data, f, ensure_ascii=False)
        final_data = []

0
1
2
3
4
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
5
6
7
8
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
9
Saving...
10
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
11
12
401 Client Error: Unauthorized for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
13
14
15
16
17
18
19
Saving...
20
21
22
23
24
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
25
26
27
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
28
401 Client Error: Unauthorized for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
29
Saving...
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed
30
401 Client Error: Unauthorized for url: https://ai.fakeopen.com/v

KeyboardInterrupt: 

In [10]:
paths = glob("./answers2/*.json")
data = []
for path in paths:
    with open(path, "r") as f:
        data.extend(json.load(f))

ds = Dataset.from_list(data)


def mapper(x):
    return {
        "messages": [
            {"role": "user", "content": x["query"]},
            {"role": "assistant", "content": x["response"]},
        ]
    }


ds = ds.map(mapper).select_columns(["messages"])
ds.to_json("./indirect_toxicity2.jsonl", force_ascii=False)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

133428

In [11]:
ds = Dataset.from_json(
    "/Users/phamhoang1408/Desktop/self-instruct/indirect_toxic/indirect_toxicity.jsonl"
)
ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 800
})

In [ ]:
ds